# Public Queue Cleaning & Standardization

This notebook cleans and standardizes the CAISO Public Queue data parsed in `01_data_inventory.ipynb`.

Goals:
- Create one canonical table with consistent columns
- Normalize project status, technology, and capacity fields
- Output `data/processed/public_queue_clean.csv`


## 1) Imports

In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

print('Executable:', sys.executable)

Executable: C:\Users\danci\Interconnection-Queue-Intelligence\.venv\Scripts\python.exe


## 2) Paths (robust to running from `notebooks/`)

In [2]:
ROOT = Path.cwd()
if ROOT.name == 'notebooks':
    ROOT = ROOT.parent

RAW = ROOT / 'data' / 'raw'
PROCESSED = ROOT / 'data' / 'processed'
PROCESSED.mkdir(parents=True, exist_ok=True)

print('ROOT:', ROOT)
print('Processed files:', [p.name for p in PROCESSED.glob('*')])

ROOT: C:\Users\danci\Interconnection-Queue-Intelligence
Processed files: ['public_queue_clean.csv', '_cluster15_raw.csv', '_cluster15_withdrawn_raw.csv', '_public_queue_raw_parsed.csv']


## 3) Load parsed public queue (from Notebook 01)

In [3]:
public_raw = pd.read_csv(PROCESSED / '_public_queue_raw_parsed.csv')
public_raw.shape

(2285, 38)

## 4) Inspect columns

We print columns once to decide what to keep.

In [4]:
for col in public_raw.columns:
    print(col)

Project Name
Queue Position
Interconnection Request
Receive Date
Queue Date
Application Status
Study
Process
Type-1
Type-2
Type-3
Fuel-1
Fuel-2
Fuel-3
MW-1
MW-2
MW-3
Net MWs to Grid
Full Capacity, Partial or Energy Only (FC/P/EO)
TPD Allocation Percentage
Off-Peak Deliverability and Economic Only
TPD Allocation Group
County
State
Utility
PTO Study Region
Station or Transmission Line
Proposed
On-line Date
(as filed with IR)
Current
On-line Date
Suspension Status
Feasibility Study or Supplemental Review
System Impact Study or 
Phase I Cluster Study
Facilities Study (FAS) or 
Phase II Cluster Study
Optional Study
(OS)
Interconnection Agreement 
Status
status
Actual
On-line Date
Project Name - Confidential
Withdrawn Date
Reason for Withdrawal


## 5) Select core columns (survivability-focused)

We keep a small, interpretable set of fields that support:
- technology categorization
- MW-weighted analysis
- basic geography/utility segmentation
- outcome status


In [5]:
CANDIDATE_COLS = [
    'Project Name',
    'Queue Position',
    'Fuel-1',
    'Fuel-2',
    'Fuel-3',
    'Net MWs to Grid',
    'County',
    'State',
    'Utility',
    'PTO Study Region',
    'Station or Transmission Line',
    'Application Status',
    'status',
]

KEEP_COLS = [c for c in CANDIDATE_COLS if c in public_raw.columns]

print('Keeping columns:')
for c in KEEP_COLS:
    print('  ✔', c)

missing = set(CANDIDATE_COLS) - set(KEEP_COLS)
if missing:
    print('\nMissing (not in this dataset):')
    for c in sorted(missing):
        print('  ✖', c)

public = public_raw[KEEP_COLS].copy()
public.shape

Keeping columns:
  ✔ Project Name
  ✔ Queue Position
  ✔ Fuel-1
  ✔ Fuel-2
  ✔ Fuel-3
  ✔ Net MWs to Grid
  ✔ County
  ✔ State
  ✔ Utility
  ✔ PTO Study Region
  ✔ Station or Transmission Line
  ✔ Application Status
  ✔ status


(2285, 13)

## 6) Standardize column names

In [6]:
public = public.rename(columns={
    'Project Name': 'project_name',
    'Queue Position': 'queue_position',
    'Fuel-1': 'fuel_primary',
    'Fuel-2': 'fuel_secondary',
    'Fuel-3': 'fuel_tertiary',
    'Net MWs to Grid': 'net_mw',
    'County': 'county',
    'State': 'state',
    'Utility': 'utility',
    'PTO Study Region': 'pto_study_region',
    'Station or Transmission Line': 'poi_station_or_line',
    'Application Status': 'application_status',
})

public.head()

,project_name,queue_position,fuel_primary,fuel_secondary,fuel_tertiary,net_mw,county,state,utility,pto_study_region,poi_station_or_line,application_status,status
0,MONTEZUMA (HIGH WINDS III),22,Wind Turbine,Battery,NaN,38.00,SOLANO,CA,PGAE,Northern,Birds Landing 230 kV,ACTIVE,active
1,TULE WIND,32,Wind Turbine,Battery,NaN,193.80,SAN DIEGO,CA,SDGE,SDGE,Boulevard East Substation 138 kV,ACTIVE,active
2,MIDWAY PEAKING,54,Natural Gas,Battery,NaN,119.90,FRESNO,CA,PGAE,Fresno,Panoche Substation,ACTIVE,active
3,FRESNO COGENERATION EXPANSION PROJECT,61,Natural Gas,Battery,NaN,73.27,FRESNO,CA,PGAE,Fresno,Helm-Kerman 70 kV Line,ACTIVE,active
4,LAKE ELSINORE ADVANCED PUMPED STORAGE PROJECT,72,Pumped-Storage hydro,NaN,NaN,500.00,RIVERSIDE,CA,SDGE,SDGE,Proposed Lee Lake Substation 500 kV,ACTIVE,active


## 7) Clean numeric fields

`net_mw` must be numeric for MW-weighted analysis.

In [7]:
public['net_mw'] = pd.to_numeric(public['net_mw'], errors='coerce')
public['net_mw'].describe()

count    2278.000000
mean      216.146401
std       270.330862
min         0.000000
25%        46.725000
50%       126.250000
75%       300.000000
max      3200.000000
Name: net_mw, dtype: float64

## 8) Technology categories

We map `fuel_primary/fuel_secondary/fuel_tertiary` into a small set of categories.
This is not perfect, but it is consistent and auditable.

In [8]:
def normalize_fuel(x):
    if pd.isna(x):
        return ''
    return str(x).strip().lower()

def infer_technology(row):
    fuels = ' '.join([
        normalize_fuel(row.get('fuel_primary')),
        normalize_fuel(row.get('fuel_secondary')),
        normalize_fuel(row.get('fuel_tertiary')),
    ])

    has_solar = ('solar' in fuels) or ('photovoltaic' in fuels) or ('pv' in fuels)
    has_storage = ('storage' in fuels) or ('battery' in fuels)
    has_wind = ('wind' in fuels)

    if has_solar and has_storage:
        return 'hybrid_solar_storage'
    if has_storage:
        return 'storage'
    if has_solar:
        return 'solar'
    if has_wind:
        return 'wind'
    return 'other'

public['technology'] = public.apply(infer_technology, axis=1)
public['technology'].value_counts(dropna=False)

technology
solar                   808
storage                 522
hybrid_solar_storage    436
other                   331
wind                    188
Name: count, dtype: int64

## 9) Outcome flags + sanity checks

In [9]:
public['queue_position'] = pd.to_numeric(public['queue_position'], errors='coerce')

public['is_withdrawn'] = (public['status'] == 'withdrawn').astype(int)
public['is_completed'] = (public['status'] == 'completed').astype(int)
public['is_active'] = (public['status'] == 'active').astype(int)

print(public['status'].value_counts(dropna=False))
public[['status','is_active','is_completed','is_withdrawn']].head()

status
withdrawn    1717
active        332
completed     236
Name: count, dtype: int64


,status,is_active,is_completed,is_withdrawn
0,active,1,0,0
1,active,1,0,0
2,active,1,0,0
3,active,1,0,0
4,active,1,0,0


## 10) Quick preview tables (counts and MW-weighted)

In [10]:
(public.groupby(['technology','status'])
       .size()
       .unstack(fill_value=0)
       .sort_values(by=['withdrawn','active','completed'], ascending=False)
)

status,active,completed,withdrawn
technology,,,
solar,6,94,708
storage,148,29,345
other,10,55,266
hybrid_solar_storage,156,38,242
wind,12,20,156


In [11]:
(public.groupby(['technology','status'])['net_mw']
       .sum()
       .unstack(fill_value=0)
       .sort_values(by=['withdrawn','active','completed'], ascending=False)
)

status,active,completed,withdrawn
technology,,,
solar,909.50,7561.8800,111183.1275
storage,36976.79,5010.7000,83279.3243
other,137.00,8780.1600,72560.5100
hybrid_solar_storage,48862.08,8645.2000,56121.0791
wind,3305.77,2934.9474,46113.4338


## 11) Save cleaned dataset

In [12]:
out_path = PROCESSED / 'public_queue_clean.csv'
public.to_csv(out_path, index=False)
out_path

WindowsPath('C:/Users/danci/Interconnection-Queue-Intelligence/data/processed/public_queue_clean.csv')